First things first, we need to import pandas.

In [1]:
import pandas as pd

Then, we have to read the data.

In [2]:
counties = pd.read_csv('data/social_capital_county.csv')

In [3]:
counties


,county,county_name,num_below_p50,pop2018,ec_county,ec_se_county,child_ec_county,child_ec_se_county,ec_grp_mem_county,ec_high_county,...,child_exposure_county,child_high_exposure_county,bias_grp_mem_county,bias_grp_mem_high_county,child_bias_county,child_high_bias_county,clustering_county,support_ratio_county,volunteering_rate_county,civic_organizations_county
0,1001,"Autauga, Alabama",5922.39210,55200.0,0.72077,0.00831,1.11754,0.02467,0.77223,1.21372,...,1.14816,1.19944,0.05526,-0.22748,0.02668,-0.08229,0.10347,0.98275,0.04355,0.01518
1,1003,"Baldwin, Alabama",15458.39600,208107.0,0.74313,0.00661,0.83064,0.01629,0.76215,1.28302,...,0.84588,1.00797,0.02950,-0.21519,0.01802,-0.05241,0.09624,0.98684,0.06117,0.01526
2,1005,"Barbour, Alabama",4863.97360,25782.0,0.41366,0.00978,0.58541,0.02707,0.35927,0.91897,...,0.63306,0.71967,0.13457,-0.34086,0.07528,-0.19714,0.14911,0.99911,0.02093,0.01474
3,1007,"Bibb, Alabama",3061.49340,22527.0,0.63152,0.01175,0.72265,0.03027,0.68094,1.06378,...,0.71433,0.72395,0.04108,-0.27727,-0.01165,-0.15993,0.14252,0.99716,0.05294,0.01439
4,1009,"Blount, Alabama",6740.91160,57645.0,0.72562,0.00985,0.76096,0.02466,0.79584,1.10569,...,0.74821,0.79375,0.00217,-0.24946,-0.01704,-0.08745,0.11243,0.99069,0.05704,0.01724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3084,56037,"Sweetwater, Wyoming",2402.96900,44117.0,0.96235,0.01280,1.14781,0.02794,1.13449,1.32399,...,1.12164,1.12907,0.09519,-0.12030,-0.02333,-0.08683,0.10809,0.99710,0.07321,0.01225
3085,56039,"Teton, Wyoming",783.24982,23059.0,1.07623,0.01744,1.23113,0.04692,1.13296,1.63551,...,1.32874,1.35341,0.14337,-0.11958,0.07346,-0.07364,0.09253,0.98648,0.09747,0.03223
3086,56041,"Uinta, Wyoming",2174.06180,20609.0,0.95452,0.01404,1.04595,0.03455,0.92831,1.32040,...,1.05446,1.06284,0.13816,-0.12194,0.00808,-0.06074,0.11204,0.99479,0.06942,0.01222
3087,56043,"Washakie, Wyoming",872.51544,8129.0,0.90667,0.01928,0.90794,0.04962,0.78223,1.29208,...,0.88480,0.88589,0.06667,-0.20435,-0.02615,-0.06076,0.11592,0.99708,0.05843,0.03512


We see that the dataset has 3089 rows. After some research, we find that we can enrich the dataset with some more counties.

So, we read some new data again.

In [4]:
all_counties = pd.read_csv('data/fips-by-state.csv',encoding = 'unicode_escape')

In [5]:
counties.rename(columns = {'county_name_x':'county_name'}, inplace = True)

In [6]:
all_counties

,fips,name,state
0,1001,Autauga County,AL
1,1003,Baldwin County,AL
2,1005,Barbour County,AL
3,1007,Bibb County,AL
4,1009,Blount County,AL
...,...,...,...
3138,56037,Sweetwater County,WY
3139,56039,Teton County,WY
3140,56041,Uinta County,WY
3141,56043,Washakie County,WY


The 'fips' column needs to be renamed to 'county' before merging. This is the column we will merge the two datasets on.

In [7]:
all_counties.rename(columns = {'fips':'county','name':'county_name'}, inplace = True)

We merge the datasets into a new one; counties_merged

In [8]:
counties_merged = pd.merge(counties,all_counties,on="county",how = 'right')

In [9]:
counties_merged.rename(columns = {'county_name_x':'county_name'}, inplace = True)

Only three columns are needed: county, county_name and ec_county.

In [10]:
counties_new = counties_merged[['county','county_name','ec_county']]

The dataset looks like this:

In [11]:
counties_new

,county,county_name,ec_county
0,1001,"Autauga, Alabama",0.72077
1,1003,"Baldwin, Alabama",0.74313
2,1005,"Barbour, Alabama",0.41366
3,1007,"Bibb, Alabama",0.63152
4,1009,"Blount, Alabama",0.72562
...,...,...,...
3138,56037,"Sweetwater, Wyoming",0.96235
3139,56039,"Teton, Wyoming",1.07623
3140,56041,"Uinta, Wyoming",0.95452
3141,56043,"Washakie, Wyoming",0.90667


County codes consist of 5 digits.
So, we have to transform the ones that have 4 digits by adding a 0 infront.

In [12]:
counties_new['county'] = counties_new['county'].apply(str)
def fixCountyCodesThatStartWithZero(x):
    if(len(x) == 4):
        return '0'+x
    else:
        return x
counties_new['county'] = counties_new['county'].apply(fixCountyCodesThatStartWithZero)

C:\Users\mernick\AppData\Local\Temp\ipykernel_5168\501655698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_new['county'] = counties_new['county'].apply(str)
C:\Users\mernick\AppData\Local\Temp\ipykernel_5168\501655698.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_new['county'] = counties_new['county'].apply(fixCountyCodesThatStartWithZero)


In [13]:
counties_new

,county,county_name,ec_county
0,01001,"Autauga, Alabama",0.72077
1,01003,"Baldwin, Alabama",0.74313
2,01005,"Barbour, Alabama",0.41366
3,01007,"Bibb, Alabama",0.63152
4,01009,"Blount, Alabama",0.72562
...,...,...,...
3138,56037,"Sweetwater, Wyoming",0.96235
3139,56039,"Teton, Wyoming",1.07623
3140,56041,"Uinta, Wyoming",0.95452
3141,56043,"Washakie, Wyoming",0.90667


Now, let's create the bins that are displayed in the legend of the wanted map. We will use the qcut method provided by pandas, which lets us create equal-sized buckets. The parameter q states how many buckets we want. Based on the map , q = 10.

In [14]:
bin_labels = pd.qcut(counties_new['ec_county'],q=10, precision=2)


In [15]:
bin_labels

0                      (0.72, 0.76]
1                      (0.72, 0.76]
2       (0.27999999999999997, 0.58]
3                      (0.58, 0.67]
4                      (0.72, 0.76]
                   ...             
3138                    (0.9, 0.97]
3139                   (1.06, 1.36]
3140                    (0.9, 0.97]
3141                    (0.9, 0.97]
3142                   (0.97, 1.06]
Name: ec_county, Length: 3143, dtype: category
Categories (10, interval[float64, right]): [(0.27999999999999997, 0.58] < (0.58, 0.67] < (0.67, 0.72] < (0.72, 0.76] ... (0.85, 0.9] < (0.9, 0.97] < (0.97, 1.06] < (1.06, 1.36]]

In [16]:
counties_new['Economic Connectedness'] = bin_labels

C:\Users\mernick\AppData\Local\Temp\ipykernel_5168\1019356067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_new['Economic Connectedness'] = bin_labels


We need to cast the bins into strings in order to remove the brackets from the intervals.

(Later, in Q3, we will use the labels argument of the qcut method to override this step)

In [17]:
counties_new['Economic Connectedness'] = counties_new['Economic Connectedness'].apply(str)

C:\Users\mernick\AppData\Local\Temp\ipykernel_5168\1714079064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_new['Economic Connectedness'] = counties_new['Economic Connectedness'].apply(str)


We create a simple function to remove the brackets from the intervals and then we apply it to the column with the bins.

In [18]:
def removeBracketsFromIntervals(x):
     return x[1:len(x)-1].replace(', ','-')
    
counties_new["Economic Connectedness"] = counties_new["Economic Connectedness"].apply(removeBracketsFromIntervals)

C:\Users\mernick\AppData\Local\Temp\ipykernel_5168\476553176.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_new["Economic Connectedness"] = counties_new["Economic Connectedness"].apply(removeBracketsFromIntervals)


So, the column looks like this.

In [19]:
counties_new['Economic Connectedness']

0                      0.72-0.76
1                      0.72-0.76
2       0.27999999999999997-0.58
3                      0.58-0.67
4                      0.72-0.76
                  ...           
3138                    0.9-0.97
3139                   1.06-1.36
3140                    0.9-0.97
3141                    0.9-0.97
3142                   0.97-1.06
Name: Economic Connectedness, Length: 3143, dtype: category
Categories (10, object): ['0.27999999999999997-0.58' < '0.58-0.67' < '0.67-0.72' < '0.72-0.76' ... '0.85-0.9' < '0.9-0.97' < '0.97-1.06' < '1.06-1.36']

In [20]:
counties_new['Economic Connectedness'] = counties_new['Economic Connectedness'].astype("string")

C:\Users\mernick\AppData\Local\Temp\ipykernel_5168\861769716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_new['Economic Connectedness'] = counties_new['Economic Connectedness'].astype("string")


In [21]:
counties_new

,county,county_name,ec_county,Economic Connectedness
0,01001,"Autauga, Alabama",0.72077,0.72-0.76
1,01003,"Baldwin, Alabama",0.74313,0.72-0.76
2,01005,"Barbour, Alabama",0.41366,0.27999999999999997-0.58
3,01007,"Bibb, Alabama",0.63152,0.58-0.67
4,01009,"Blount, Alabama",0.72562,0.72-0.76
...,...,...,...,...
3138,56037,"Sweetwater, Wyoming",0.96235,0.9-0.97
3139,56039,"Teton, Wyoming",1.07623,1.06-1.36
3140,56041,"Uinta, Wyoming",0.95452,0.9-0.97
3141,56043,"Washakie, Wyoming",0.90667,0.9-0.97


However, we need to handle the NA values accordingly.

Since the qcut method ignores the NA values, we need to take care of that on our own.

We stringify the NA and insert 'NA' into the column of the corresponding counties.

In [22]:
counties_new['Economic Connectedness'] = counties_new['Economic Connectedness'].fillna("NA")

C:\Users\mernick\AppData\Local\Temp\ipykernel_5168\891845931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_new['Economic Connectedness'] = counties_new['Economic Connectedness'].fillna("NA")


In [23]:
counties_new['Economic Connectedness'] = counties_new['Economic Connectedness'].astype("string")

C:\Users\mernick\AppData\Local\Temp\ipykernel_5168\861769716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_new['Economic Connectedness'] = counties_new['Economic Connectedness'].astype("string")


We need to change the first and the last interval according to the map.

In [24]:
counties_new.loc[counties_new['Economic Connectedness'] == '0.27999999999999997-0.58', 'Economic Connectedness'] = '< 0.58'

In [25]:
counties_new['Economic Connectedness']

0       0.72-0.76
1       0.72-0.76
2          < 0.58
3       0.58-0.67
4       0.72-0.76
          ...    
3138     0.9-0.97
3139    1.06-1.36
3140     0.9-0.97
3141     0.9-0.97
3142    0.97-1.06
Name: Economic Connectedness, Length: 3143, dtype: string

In [26]:
counties_new.loc[counties_new['Economic Connectedness'] == '1.06-1.36', 'Economic Connectedness'] = '> 1.06'

In [27]:
counties_new

,county,county_name,ec_county,Economic Connectedness
0,01001,"Autauga, Alabama",0.72077,0.72-0.76
1,01003,"Baldwin, Alabama",0.74313,0.72-0.76
2,01005,"Barbour, Alabama",0.41366,< 0.58
3,01007,"Bibb, Alabama",0.63152,0.58-0.67
4,01009,"Blount, Alabama",0.72562,0.72-0.76
...,...,...,...,...
3138,56037,"Sweetwater, Wyoming",0.96235,0.9-0.97
3139,56039,"Teton, Wyoming",1.07623,> 1.06
3140,56041,"Uinta, Wyoming",0.95452,0.9-0.97
3141,56043,"Washakie, Wyoming",0.90667,0.9-0.97


We see that the economic connectedness, when hovering over a county on the map, has 2 decimal points.

So, we have to round the column 'ec_county' to 2 decimal points. 

We will use the round method.

In [28]:
counties_new['ec_county'] = round(counties_new['ec_county'],2)

C:\Users\mernick\AppData\Local\Temp\ipykernel_5168\3054324511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_new['ec_county'] = round(counties_new['ec_county'],2)


In [29]:
# counties_new['economic connectedness'] = counties_new['economic connectedness'].fillna("NA")

In order to plot the counties, we need their coordinates on the map. 

After some research online, we read the coordinates of the counties and assign the dataset into the variable 'counties_json'.

In [30]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties_json = json.load(response)
    

Finally, before plotting the map, we need to import some libraries.

In [31]:
import matplotlib.pyplot as plt
from matplotlib.colorbar import ColorbarBase

import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature 

from metpy.plots import USCOUNTIES
from metpy.cbook import get_test_data

import plotly.express as px
import plotly.figure_factory as ff


C:\Users\mernick\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Note that the map will be displayed in a file named 'Economic Connectedness.html', which will be created in the current folder of this file.

In [39]:
fig = px.choropleth(data_frame = counties_new,
                    geojson=counties_json,
                    hover_name = 'county_name',
                    locations='county',
                    color='Economic Connectedness',
                    scope="usa",
                    category_orders = {'Economic Connectedness':[
                                                'NA',
                                                '< 0.58',
                                                 '0.58-0.67',
                                                 '0.67-0.72',
                                                 '0.72-0.76',
                                                 '0.76-0.81',
                                                 '0.81-0.85',
                                                 '0.85-0.9',
                                                  '0.9-0.97',
                                                  '0.97-1.06',
                                                  '> 1.06'
                                                 ]},
                    labels = {'ec_county':'economic connectedness'},
                    hover_data = {'ec_county':True,
                                 'Economic Connectedness':False},
                    color_discrete_map = {
                        'NA':"rgb(255,215,0)",
                        '< 0.58':"rgb(178,24,43)",
                        '0.58-0.67':"rgb(235,176,166)",
                        '0.67-0.72':"rgb(244,165,130)",
                        '0.72-0.76':"rgb(253,219,199)",
                        '0.76-0.81':"rgb(247,247,247)",
                        '0.81-0.85':"rgb(209,229,240)",
                        '0.85-0.9':"rgb(146,197,222)",
                        '0.9-0.97':"rgb(67,147,195)",
                        '0.97-1.06':"rgb(33,102,172)",
                        '> 1.06':"rgb(5,48,97)"
                    }
                    )
fig.update_layout(legend=dict(
    yanchor = "top",
    xanchor = "right",
    y=0.8,
    x=1
))
fig.update_layout(legend_traceorder="reversed")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html('Economic Connectedness.html', auto_open=True)